# Import Modules

In [10]:
#It is important the DCS computers have all the following modules installed in order for this code to run
#Most notably: keras (most recent version), tensorflow, pathlib are required

import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf
from keras.layers import merge
from keras.models import Model
from keras.models import Sequential
from keras.layers import BatchNormalization, Deconvolution2D, Conv1D, Conv2D, Conv2DTranspose, UpSampling2D, Lambda, Dense, Activation, Input, Dropout, MaxPooling2D, concatenate
from keras import backend as K
from keras import optimizers

#Others
%matplotlib inline
import pandas as pd
from skimage.filters import threshold_otsu
from scipy import ndimage
import pathlib
import imageio
import numpy as np
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
import skimage
from scipy import ndimage as ndi
from skimage.morphology import watershed
from skimage.feature import peak_local_max
from sklearn.model_selection import GridSearchCV
from numpy import random
from sklearn.neural_network import MLPClassifier
from skimage.filters import threshold_otsu
import cv2

# Parameters and Feature Engineering Switches

In [ ]:
#Parameters
MIN_SIZE = 10         #Delete cell masks smaller than this
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3
TRAIN_PATH = '/modules/cs342/Assignment2/FullTesting/'
TEST_PATH = '/modules/cs342/Assignment2/FullTesting/'

#Feature Engineering (do not use hog and channel scaling simulatenously - we could modify the script to include both original image and feature engineered in training set easily)
otsu = False            #Whether to use otsu or just round up/down from 0.5
ws = False              #Whether to use watershed
hog = False             #Whether to use histogram of orientated gradients
CHANNEL_SCALING = True  #Whether to use channel scaling

# Import Data and Rescaling/Augmenting

In [6]:
#Channel Scale the Images
def scale_img_channels(img):
    #Scales the image channels linearly to use the entire range of 0 to 255
    #img: the 4 channel image we are rescaling
    #returns: the channel rescaled image
    for i in range(IMG_CHANNELS):
        channel = img[:,:,i]
        channel = channel - channel.min()
        channelmax = channel.max()
        if channelmax > 0:
            factor = 255/channelmax
            channel = (channel * factor).astype(int)
        img[:,:,i] = channel
    return img

#HoG
def HoG(img):
    #calculates the histogram of orientated gradients
    #image: the image we wish to calculated on
    #returns: the magnitude of the x and y gradients
    img = rgb2gray(img)
    img = np.float32(img) / 255.0
    # Calculate gradient 
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0, ksize=1)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1, ksize=1)
    # Python Calculate gradient magnitude and direction (in degrees) 
    mag, angle = cv2.cartToPolar(gx, gy, angleInDegrees=True)
    return mag


#THE FOLLWOING IS MOSTLY A COPY OF Kjetil Åmdal-Sævik's NOTEBOOK: https://www.kaggle.com/keegil/keras-u-net-starter-lb-0-277

# Get train and test IDs
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]

# Get and resize train images and masks
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
print('Getting and resizing train images and masks ... ')
if CHANNEL_SCALING == True:
    print('... and channel scaling ... ')
if hog == True:
    print('... and HoG-ing ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = TRAIN_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    if CHANNEL_SCALING == True:
        img = scale_img_channels(img)
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_train[n] = img
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                      preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)
    Y_train[n] = mask

# Get and resize test images
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []
print('Getting and resizing test images ... ')
if CHANNEL_SCALING == True:
    print('... and channel scaling ... ')
if hog == True:
    print('... and HoG-ing ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    if CHANNEL_SCALING == True:
        img = scale_img_channels(img)
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img

    
print('Done!')

Getting and resizing train images and masks ... 
... and channel scaling ... 


100%|██████████| 670/670 [01:38<00:00,  6.80it/s]

Getting and resizing test images ... 
... and channel scaling ... 



100%|██████████| 65/65 [00:01<00:00, 62.15it/s]

Done!


# Loss Functions and Metrics

In [7]:
# Define IoU metric
#Reference: cpmpml's modification of Kjetil Åmdal-Sævik's metric (https://www.kaggle.com/keegil/keras-u-net-starter-lb-0-277)
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2, y_true)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

# Defining Inverse Dice Function
#Reference: Kevin Mader (https://www.kaggle.com/kmader/nuclei-overview-to-submission)
smooth = 1.
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

# U-Net Network Architecture

In [ ]:
# Build U-Net model
#Reference: Kjetil Åmdal-Sævik's implementation (https://www.kaggle.com/keegil/keras-u-net-starter-lb-0-277) of the paper (https://arxiv.org/pdf/1505.04597.pdf)

inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])

model.summary()

# Compiling and Fitting Model

In [12]:
#CHOICE OF HYPER PARAMETERS ARE DUE TO Raoul Malm: (https://www.kaggle.com/raoulma/nuclei-dsb-2018-tensorflow-u-net-score-0-352)

# Make a new optimiser
opt = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.1, amsgrad=False)

# Compile model
model.compile(optimizer=opt, loss=dice_coef_loss, metrics=[mean_iou])

#Fit Model
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('q3model.h5', verbose=1, save_best_only=True)
results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=16, epochs=30, callbacks=[earlystopper, checkpointer])

KeyboardInterrupt: 

# Check Predictions

In [ ]:
#Display the probablistic mask given by our MLP and the actual image for a random training image

#Load model
model = load_model('q3model.h5', custom_objects={'mean_iou': mean_iou, 'dice_coef_loss': dice_coef_loss})

#Random index
n = random.randint(len(X_train_pixels))

#Get masks
a = model.predict(X_train_pixels)[n]
b = Y_train_pixels[n]

#Reshape and display
a=a.reshape((IMG_HEIGHT,IMG_WIDTH))
b=b.reshape((IMG_HEIGHT,IMG_WIDTH))
plt.imshow(a)
plt.imshow(b)

# Running Test Data and RLE Encoding

In [ ]:
#Watershed Segmentation
def watershed_segmentation_dist(labels,display=False):
    
    #performs watershed segmentation where the distance function is the distance to the edge of the label
    
    #labels: an array of numbers, 0 value for background and value i for the i^th label
    #display: gives a picture of our outputs
    
    #returns: new labels for the image after being watershed segmented
    
    distance = ndi.distance_transform_edt(labels)
    local_maxi = peak_local_max(distance, indices=False, labels=labels)
    markers = ndi.label(local_maxi)[0]
    labels = watershed(-distance, markers, mask=labels)
    if display == True:
        fig, axes = plt.subplots(ncols=3, figsize=(9, 3), sharex=True, sharey=True,
                                 subplot_kw={'adjustable': 'box-forced'})
        ax = axes.ravel()
        ax[0].imshow(labels, cmap=plt.cm.gray, interpolation='nearest')
        ax[0].set_title('Overlapping objects')
        ax[1].imshow(-distance, cmap=plt.cm.gray, interpolation='nearest')
        ax[1].set_title('Distances')
        ax[2].imshow(labels, cmap=plt.cm.spectral, interpolation='nearest')
        ax[2].set_title('Separated objects')
        for a in ax:
            a.set_axis_off()
        fig.tight_layout()
        plt.show()
    return labels


#Reference: https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
def rle_encoding(x):
    
    #Encodes the label into RLE
    
    #x: numpy array of shape (height, width), 1 - mask, 0 - background
    
    #returns: RLE encoding of the label
    
    dots = np.where(x.T.flatten()==1)[0] # .T sets Fortran order down-then-right
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b+1, 0))
        run_lengths[-1] += 1
        prev = b
    return " ".join([str(i) for i in run_lengths])


def RLE_mask(mask,image_id):
    '''
    Returns pandas dataframe of each RLE string for each (predicted) nuclei of an image
    '''
    # Deriving individual mask for each object
    labels, numlabels = ndimage.label(mask)           # Labels each component with different number
    if ws == True:
        labels = watershed_segmentation_dist(labels,display=False)     # Watershed
        numlabels = np.amax(labels)+1                 # Plus one due to zero indexing
    label_arrays = []
    for n in range(1, numlabels+1, 1):
        label_mask = np.where(labels == n, 1, 0)
        if sum(sum(label_mask)) >= MIN_SIZE:          # Checks if (predicted) nuclei is big enough to count
            label_arrays.append(label_mask)
        else:
            mask = np.where(labels == n, 0, mask)
    labels, numlabels = ndimage.label(mask)           # Regenerate labels

    #Writing image_df to be added to the main df
    im_df = pd.DataFrame(columns=["ImageID","EncodedPixels"])
    for n in range(1, numlabels+1, 1):
            label_mask = np.where(labels == n, 1, 0)
            rle_string = rle_encoding(label_mask)
            series = pd.Series({'ImageID': image_id, 'EncodedPixels': rle_string})
            im_df = im_df.append(series, ignore_index=True)
    return im_df


#Load model
model = load_model('q3model.h5', custom_objects={'mean_iou': mean_iou, 'dice_coef_loss': dice_coef_loss})

#predicted test values due to our model
preds_test = model.predict(X_test, verbose=1)

# Resize our predicted test values to their original image sizes
preds_test_resized = []
for i in range(len(preds_test)):
    preds_test_resized.append(resize(np.squeeze(preds_test[i]), (sizes_test[i][0], sizes_test[i][1]), mode='constant', preserve_range=True))

# This will be our final dataframe
df = pd.DataFrame(columns=["ImageID","EncodedPixels"])

j=0
# RLE encode the test data
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    image_id = id_
    image_label = preds_test_resized[j]
    if otsu == True:
        thresh_val = threshold_otsu(image_label)
    else:
        thresh_val = 0.5
    newmask = np.where(image_label > thresh_val, 1, 0)
    add_to_df = RLE_mask(newmask,image_id)
    df = df.append(add_to_df, ignore_index=True)
    j+=1
    
df.to_csv('Q3submission.csv', index=False)